# Homework 3: Logistics Regression
Implement everywhere that has an ellipsis (`...`).

In [1]:
import numpy as np

### Load dataset

In [2]:
from sklearn.datasets import load_iris
data = load_iris()
X, Y = data.data, data.target
print(X.shape,Y.shape)


(150, 4) (150,)


In [3]:
# only classify class 1 vs. class 2
index = np.where(Y == 0)
Y = np.delete(Y,index)
for i in index:
    X = np.delete(X,(i),axis = 0)
# idxs = Y > 0
# X = X[idxs, :]
# Y = Y[idxs]

In [ ]:
idxs = Y > 0
A = X[idxs, :]
B = Y[idxs]
x_mean = np.mean(A, axis = 0)
x_std = np.std(A, axis = 0)
A = (A - x_mean)/x_std
B -= 1
# X = np.concatenate((np.ones((1, X.shape[1])), X), axis = 0)
A = np.hstack([np.ones((A.shape[0], 1)), A])
print(B)
print(A)

### Data normalization
Remember: normalization is done per feature, not all at once. Also, remember that Y is [1, 2] instead of [0, 1].

In [4]:
x_mean = np.mean(X, axis = 0)
x_std = np.std(X, axis = 0)
X = (X - x_mean)/x_std
Y -= 1
# X = np.concatenate((np.ones((1, X.shape[1])), X), axis = 0)
X = np.hstack([np.ones((X.shape[0], 1)), X])
print(Y)
print(X)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[[ 1.          1.11900931  0.99068792 -0.25077906 -0.65303909]
 [ 1.          0.20924564  0.99068792 -0.49425387 -0.41643072]
 [ 1.          0.96738203  0.68864892 -0.00730424 -0.41643072]
 [ 1.         -1.15539985 -1.72766308 -1.10294091 -0.88964745]
 [ 1.          0.36087292 -0.21746808 -0.37251647 -0.41643072]
 [ 1.         -0.8521453  -0.21746808 -0.49425387 -0.88964745]
 [ 1.          0.05761837  1.29272692 -0.25077906 -0.17982236]
 [ 1.         -2.06516352 -1.42562408 -1.95510276 -1.59947255]
 [ 1.          0.5125002   0.08457092 -0.37251647 -0.88964745]
 [ 1.         -1.61028169 -0.51950708 -1.22467832 -0.65303909]
 [ 1.         -1.91353624 -2.63378009 -1.71162795 -1.59947255]
 [ 1.         -0.54889074  0.38660992 -0.8594661  -0.41643072]
 [ 1.         -0.39726347 -2.02970209 -1

In [ ]:
pham.thi.hong.anh@sun-asterisk.com

In [ ]:
np.array_equiv(A,X)

### Define classification model
Create parameters here. Initialize with zeros. In case you forgot: $Y = \sigma(X\Theta)$.

In [5]:
weight = np.zeros((X.shape[1]))

### Define evaluation metrics

In [6]:
def accuracy(X, Y, weight):
    '''
    Evaluate the model, represented by `weight`, with data (X, Y).
    
    Input:
        X:      data features
        Y:      data labels
        weight: model weights
    Ouput:
        Model accuracy on input data.
    '''
    result = (X @ weight > 0) == Y
    return np.sum(result) / result.size
    

### Hyperparameters

In [7]:
# learning rate
alpha = 1e-2
# epochs
epoch = 10000

### Train the model.

In [8]:
def sigmoid(X, weight):
    return 1 / (1 + np.exp(-X @ weight))
def loss(X, Y, sigmoid):
    sigmoid = sigmoid(X,weight)
    return -np.sum(Y * np.log(sigmoid),(1-Y) * np.log(1-sigmoid))/ sigmoid.size
for ep in range(0, epoch):
    loss = loss(X, Y, sigmoid)
    gradient = X.T @ (sigmoid(X,weight) - Y) / sigmoid(X, weight).size
    weight -= alpha * gradient
    acc = accuracy(X, Y, weight)

print(weight)

[ 0.12047504 -0.44156746 -0.89309501  2.965364    3.3427994 ]


In [9]:
# just to double check
assert np.allclose(weight, np.array([0.12047504, -0.44156746, -0.89309501, 2.965364, 3.3427994]))

In [10]:
# model accuracy
accuracy(X, Y, weight)

0.96

### (Optional) How low can you go?
Do anything you want to get the best performance out of the training set. For once, let's overfit to your heart's content.

In [ ]:
# do some fun code here and try to match this :)
...

In [ ]:
print('Weights:', weight)
y = 1 / (1 + np.exp(-X @ weight))
loss = -np.sum(Y * np.log(y) + (1 - Y) * np.log(1 - y)) / y.size
print('Loss:', loss)
print('Accuracy:', accuracy(X, Y, weight))